# Computer Vision

<p>Computer vision(bilgisayar görüşü), bilgisayarların dijital görüntülerle çalışarak çevresel bilgileri anlama ve yorumlama yeteneği olarak tanımlanabilir. Bu alan, görüntü veya video verilerini işleyen algoritmalar ve sistemler geliştirmeyi içerir. Temel amaç, bilgiyarların görüntü verilerini anlamasını, nesneleri tanımasını, sahneleri anlamasını ve belki insan benzeri görme yeteneklerini taklit etmesini sağlamaktır.</p>
<p>Computer vision, birçok farklı uygulama alanında kullanılabilir. Bunlar arasında:</p>
<ol>
<li><strong>Nesne Tanıma ve Sınıflandırma: </strong>Bilgisayarların görüntülerdeki nesneleri tanıması ve sınıflandırması. Örneğin, bir görüntüdeki arabaları, insanları veya evcil hayvanları tanıma.</li>
<br>
<li><strong>Yüz Tanıma: </strong>Görüntülerdeki insan yüzlerini tanıma ve hatta belirli bireyleri tanıma.</li>
<br>
<li><strong>Hareket Takibi: </strong>Video görüntülerindeki hareketli nesneleri takip etme ve analiz etme. Bu, güvenlik sistemlerinde, trafik izleme sistemlerinde ve oyunlarda kullanılabilir.</li>
<br>
<li><strong>Görüntü Düzenleme ve İyileştirme: </strong>Görüntülerdeki gürültüyü azaltma, keskinleştirme, renk düzeltme gibi işlemler yapma.</li>
</ol>
<p>Bunlar sadece birkaç örnek; computer vision teknolojileri daha birçok alanda yaygın olarak kullanılmaktadır.</p>

### 0- Kütüphanenin kurulumu

In [1]:
#pip install opencv-python

### 1- Nesne Tanıma ve Sınıflandırma

In [1]:
import cv2
import numpy as np

# Webcam'den görüntü almak için VideoCapture nesnesini başlatma
cap = cv2.VideoCapture(0)

while True:
    # Kameradan bir kare al
    ret, frame = cap.read()
    
    # Görüntüyü HSV renk uzayına dönüştürme
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Mavi renk için bir renk aralığı belirleme (örneğin, mavi bir top tespit etmek için)
    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])
    
    # Belirlenen renk aralığında maskeleme işlemi
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # Maske üzerinde kontur tespiti
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Konturları çizme
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Sonuçları gösterme
    cv2.imshow('Object Detection', frame)
    
    # 'q' tuşuna basıldığında döngüyü sonlandırma
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Video yakalama nesnesini ve pencereyi serbest bırakma
cap.release()
cv2.destroyAllWindows()


### 2- Yüz Tanıma

In [2]:
import cv2

# Haar yüz sınıflandırıcısını yükleme
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Webcam'den görüntü almak için VideoCapture nesnesini başlatma
cap = cv2.VideoCapture(0)

while True:
    # Kameradan bir kare al
    ret, frame = cap.read()
    
    # Görüntüyü griye dönüştürme (hızlı işlem için)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Yüzleri tespit etme
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Tespit edilen yüzlerin etrafına dikdörtgen çizme
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Sonuçları gösterme
    cv2.imshow('Face Detection', frame)
    
    # 'q' tuşuna basıldığında döngüyü sonlandırma
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Video yakalama nesnesini ve pencereyi serbest bırakma
cap.release()
cv2.destroyAllWindows()

### 3- Hareket Takibi

In [3]:
import cv2

# Webcam'den görüntü almak için VideoCapture nesnesini başlatma
cap = cv2.VideoCapture(0)

# İlk kareyi okuma
ret, frame1 = cap.read()
# İlk kareyi gri tonlamaya dönüştürme
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

while True:
    # Yeni bir kare okuma
    ret, frame2 = cap.read()
    # Yeni kareyi gri tonlamaya dönüştürme
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Ardışık kareler arasındaki piksel farkını hesaplama
    diff = cv2.absdiff(gray1, gray2)
    # Piksel farkını eşikleme işlemi
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    # Gürültüyü azaltma için erozyon ve genişletme işlemleri
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    # Hareketli nesnelerin konturlarını bulma
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Hareketli nesnelerin etrafına dikdörtgen çizme
    for contour in contours:
        if cv2.contourArea(contour) > 1000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame2, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Sonuçları gösterme
    cv2.imshow('Motion Tracking', frame2)
    
    # Ardışık kareleri güncelleme
    gray1 = gray2
    
    # 'q' tuşuna basıldığında döngüyü sonlandırma
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Video yakalama nesnesini ve pencereyi serbest bırakma
cap.release()
cv2.destroyAllWindows()

### 4- Görüntü Düzenleme ve İyileştirme

In [6]:
import cv2
import numpy as np

# Görüntüyü yükleme
image = cv2.imread('bio_2.jpg')

# Görüntüyü gri tonlamaya dönüştürme
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Görüntüyü yeniden boyutlandırma
resized_image = cv2.resize(image, (500, 300))

# Görüntüyü döndürme
(h, w) = image.shape[:2]
center = (w // 2, h // 2)
rotation_matrix = cv2.getRotationMatrix2D(center, 45, 1.0)
rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))

# Görüntüyü aynalama
flipped_image = cv2.flip(image, 1)

# Görüntüyü bulanıklaştırma
blurred_image = cv2.GaussianBlur(image, (15, 15), 0)

# Görüntüyü HSV formatına dönüştürme
hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Sonuçları görselleştirme
cv2.imshow('Original Image', image)
cv2.imshow('Gray Image', gray_image)
cv2.imshow('Resized Image', resized_image)
cv2.imshow('Rotated Image', rotated_image)
cv2.imshow('Flipped Image', flipped_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow("HSV Tonlama", hsv_img[:,:,:])
cv2.imshow("Hue Kanali", hsv_img[:,:,0])
cv2.imshow("Saturation Kanali", hsv_img[:,:,1])
cv2.imshow("Value Kanali", hsv_img[:,:,2])

# 'q' tuşuna basıldığında döngüyü sonlandırma
cv2.waitKey(0)
cv2.destroyAllWindows()